# DataFrame Pre Processing

In [2]:
import pandas as pd
import numpy as np

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df = pd.read_html(url)[0]
df.drop(df.loc[df['Borough'] == 'Not assigned'].index, inplace = True)
df.reset_index(drop = True, inplace = True)

In [17]:
d_flag = False
for i,a in enumerate(df['Postal code']):
    for j,b in enumerate(df['Postal code']):
        if j > i:
            if a==b:
                print(a,b,'are duplicates')
                d_flag = True
if d_flag == False:
    print('No duplicates in the dataframe')

No duplicates in the dataframe


In [5]:
df.loc[df['Postal code'] == 'M5A']

,Postal code,Borough,Neighborhood
2,M5A,Downtown Toronto,Regent Park / Harbourfront


Neighborhoods are already combined but splitted by "/" instead of ","

In [16]:
df['Neighborhood'] = df['Neighborhood'].str.replace('/',',')

In [33]:
print(df[['Neighborhood']].isna().values.any())
print(df.isna().values.any())
print((df['Neighborhood'] == 'Not assigned').values.any())

False
False
False


No NAN or Not assigned values

In [70]:
print(df.shape)

(103, 5)


# Adding Latitude and Longitude

In [41]:
latlon = pd.read_csv('Geospatial_Coordinates.csv')

In [67]:
# create lat and lon columns in df 
df['Latitude'] = np.nan
df['Longitude'] = np.nan
df.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,NaN,NaN
1,M4A,North York,Victoria Village,NaN,NaN
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",NaN,NaN
3,M6A,North York,"Lawrence Manor , Lawrence Heights",NaN,NaN
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",NaN,NaN


Assign values from latlot dataframe to df 

In [68]:
for i, row in df.iterrows():
    df.at[i,'Latitude']=latlon.loc[latlon['Postal Code'] == row['Postal code']].Latitude
    df.at[i,'Longitude'] = latlon.loc[latlon['Postal Code'] == row['Postal code']].Longitude

In [69]:
df.head(10)

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill , Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


# Explore Neighborhoods

In [96]:
from geopy.geocoders import Nominatim
import folium
import json
import requests

In [78]:
address = 'Toronto, CA'
geolocator = Nominatim(user_agent = 'toronto_explorer')
location= geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

In [163]:
map_toronto = folium.Map(location = [latitude, longitude], zoom_start = 10 )
for lat, lon, label in zip(df['Latitude'],df['Longitude'],df['Neighborhood']):
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
        [lat,lon],
        radius = 5,
        popup = label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity = 0.7,
        parse_html=False
    ).add_to(map_toronto)
map_toronto

In [93]:
#initializing foursquare login data
CLIENT_ID = *******************************
CLIENT_SECRET = *****************************
VERSION = '20200410'
LIMIT = 100

In [94]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [97]:
toronto_venues = getNearbyVenues(names = df['Neighborhood'],
                                latitudes = df['Latitude'],
                                longitudes = df['Longitude'])

Parkwoods
Victoria Village
Regent Park , Harbourfront
Lawrence Manor , Lawrence Heights
Queen's Park , Ontario Provincial Government
Islington Avenue
Malvern , Rouge
Don Mills
Parkview Hill , Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park , Princess Gardens , Martin Grove , Islington , Cloverdale
Rouge Hill , Port Union , Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate , Bloordale Gardens , Old Burnhamthorpe , Markland Wood
Guildwood , Morningside , West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor , Wilson Heights , Downsview North
Thorncliffe Park
Richmond , Adelaide , King
Dufferin , Dovercourt Village
Scarborough Village
Fairview , Henry Farm , Oriole
Northwood Park , York University
East Toronto
Harbourfront East , Union Station , Toronto Islands
Little Portugal , Trinity
Kennedy Park , Ionview , East Birchmount Park
Bayview Village
Do

In [103]:
print(toronto_venues.shape)
toronto_venues.groupby('Neighborhood').count()
print('There are {} unique categories.'.format(len(toronto_venues['Venue Category'].unique())))

(2173, 7)
There are 267 unique categories.


In [108]:
# one hot encoding of venue categories
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix = '', prefix_sep = '' )
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood']
fixed_columns = ['Neighborhood'] + toronto_onehot.columns.drop(['Neighborhood']).tolist()
toronto_onehot = toronto_onehot[ fixed_columns  ]
print(toronto_onehot.shape)
toronto_onehot.head()

(2173, 267)


,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [129]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
print(toronto_grouped.shape)
i=0
for hood in toronto_grouped['Neighborhood']:
    i = i+1
    temp =toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq':2})
    print('----',hood,'----')
    print(temp.sort_values('freq', ascending = False).reset_index(drop=True).head(5))
    print('\n')
    if i >=3:
        break # break loop after first 3 neighborhoods

(95, 267)
---- Agincourt ----
                       venue  freq
0               Skating Rink  0.25
1                     Lounge  0.25
2             Breakfast Spot  0.25
3  Latin American Restaurant  0.25
4         Mexican Restaurant  0.00


---- Alderwood , Long Branch ----
            venue  freq
0     Pizza Place  0.18
1    Dance Studio  0.09
2        Pharmacy  0.09
3             Gym  0.09
4  Sandwich Place  0.09


---- Bathurst Manor , Wilson Heights , Downsview North ----
                       venue  freq
0                       Bank  0.10
1                Coffee Shop  0.10
2                Bridal Shop  0.05
3  Middle Eastern Restaurant  0.05
4        Fried Chicken Joint  0.05




In [130]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [131]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Latin American Restaurant,Skating Rink,Breakfast Spot,Lounge,Yoga Studio,Donut Shop,Discount Store,Distribution Center,Dog Run,Doner Restaurant
1,"Alderwood , Long Branch",Pizza Place,Gym,Pub,Athletics & Sports,Sandwich Place,Pool,Dance Studio,Pharmacy,Coffee Shop,Skating Rink
2,"Bathurst Manor , Wilson Heights , Downsview North",Coffee Shop,Bank,Fried Chicken Joint,Bridal Shop,Sandwich Place,Diner,Restaurant,Ice Cream Shop,Supermarket,Sushi Restaurant
3,Bayview Village,Café,Bank,Chinese Restaurant,Japanese Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Yoga Studio
4,"Bedford Park , Lawrence Manor East",Restaurant,Sandwich Place,Italian Restaurant,Coffee Shop,Liquor Store,Pharmacy,Café,Sushi Restaurant,Pub,Butcher


# Run KMeans Clustering

In [149]:
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

In [142]:
k=5
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood',1)
kmeans = KMeans(n_clusters = k, random_state = 42).fit(toronto_grouped_clustering)
kmeans.labels_[0:300]
neighborhoods_venues_sorted.insert(0, 'Cluster Label', kmeans.labels_)

In [143]:
neighborhoods_venues_sorted.head()

,Cluster Label,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,Agincourt,Latin American Restaurant,Skating Rink,Breakfast Spot,Lounge,Yoga Studio,Donut Shop,Discount Store,Distribution Center,Dog Run,Doner Restaurant
1,0,"Alderwood , Long Branch",Pizza Place,Gym,Pub,Athletics & Sports,Sandwich Place,Pool,Dance Studio,Pharmacy,Coffee Shop,Skating Rink
2,0,"Bathurst Manor , Wilson Heights , Downsview North",Coffee Shop,Bank,Fried Chicken Joint,Bridal Shop,Sandwich Place,Diner,Restaurant,Ice Cream Shop,Supermarket,Sushi Restaurant
3,0,Bayview Village,Café,Bank,Chinese Restaurant,Japanese Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Yoga Studio
4,0,"Bedford Park , Lawrence Manor East",Restaurant,Sandwich Place,Italian Restaurant,Coffee Shop,Liquor Store,Pharmacy,Café,Sushi Restaurant,Pub,Butcher


In [161]:
toronto_merged = df.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
toronto_merged.dropna(subset=['Cluster Label'],axis = 0, inplace = True)
toronto_merged['Cluster Label'] = toronto_merged['Cluster Label'].astype(int)

In [164]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Label']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=8,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters